## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'STATUS'])

In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

In [ ]:
# application_df = application_df[application_df['INCOME_AMT'] != '0']
value_counts = application_df['NAME'].value_counts()
value_counts[value_counts > 1].head(30)

In [ ]:
names_to_replace = value_counts[value_counts < 100].index.tolist()
names_to_replace

In [ ]:
# Replace in dataframe
for name in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name,"Other")

# Check to make sure replacement was successful
application_df['NAME'].value_counts()

In [ ]:
application_df

In [ ]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
value_counts = application_df['APPLICATION_TYPE'].value_counts()
value_counts

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = value_counts[value_counts < 5].index.tolist()
application_types_to_replace

In [ ]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
value_counts = application_df['CLASSIFICATION'].value_counts()

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
value_counts[value_counts > 1]

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = value_counts[value_counts < 50].index.tolist()
classifications_to_replace


In [ ]:

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

In [ ]:
application_df.describe()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
df_dummies = pd.get_dummies(application_df, columns=['NAME', 
                                                     'APPLICATION_TYPE', 
                                                     'AFFILIATION', 
                                                     'CLASSIFICATION', 
                                                     'USE_CASE', 
                                                     'ORGANIZATION', 
                                                     'INCOME_AMT', 
                                                     'SPECIAL_CONSIDERATIONS'])
df_dummies.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = df_dummies['IS_SUCCESSFUL'].values
X = df_dummies.drop('IS_SUCCESSFUL', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=8)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()
# scaler = RobustScaler()
# scaler = MinMaxScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 4
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 16


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                          input_dim=number_input_features, 
                          activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, 
                             activation="relu"))
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, 
#                              activation="relu"))
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, 
#                              activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1,
                             activation="sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=20)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn.save('model_2layer.h5')